# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES(x0=np.random.random(problem.n_var))

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")


Best solution found: 
X = [0.50000003 0.49999996 0.49999996 0.5        0.49999996 0.49999995
 0.49999995 0.5        0.49999994 0.50000006]
F = [1.78865908e-14]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.594179281 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.418903173 |  0.097485873 |  0.09382 |  0.10175 |  1.21430
    3 |      21 |  0.321205163 |  0.103770490 |  0.09812 |  0.11014 |  1.27118
    4 |      31 |  0.318396788 |  0.109029295 |  0.10258 |  0.11725 |  1.40985
    5 |      41 |  0.318396788 |  0.114395048 |  0.10740 |  0.12788 |  1.54345
    6 |      51 |  0.225561272 |  0.109222313 |  0.10200 |  0.12225 |  1.56851
    7 |      61 |  0.183858735 |  0.113533576 |  0.10492 |  0.12126 |  1.54265
    8 |      71 |  0.109093868 |  0.120252305 |  0.10999 |  0.12893 |  1.63895
    9 |      81 |  0.109093868 |  0.118561898 |  0.10715 |  0.12784 |  1.68209
   10 |      91 |  0.047290280 |  0.122470685 |  0.10895 |  0.13523 |  1.76182
Best solution found: 
X = [0.49974691 0.46937795 0.5043571  0.52346965 0.39020405 0.50208668
 0.52311227 0.4581065  0.66924532 0.44

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.594179281 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.443917487 |  0.088753927 |  0.08568 |  0.08996 |  1.15887
    3 |      21 |  0.353209382 |  0.092350482 |  0.08746 |  0.09629 |  1.29570
    4 |      31 |  0.342124547 |  0.097598831 |  0.09148 |  0.10365 |  1.38505
    5 |      41 |  0.306768146 |  0.100318584 |  0.09283 |  0.10866 |  1.49728
    6 |      51 |  0.205397570 |  0.100784525 |  0.09294 |  0.11153 |  1.59194
Best solution found: 
X = [0.56858431 0.62869872 0.64707866 0.58311253 0.27306507 0.59385726
 0.55136856 0.72622116 0.62083624 0.33609212]
F = [0.20539757]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [ 6.01821704e-10 -3.76180864e-10]
F = [0.]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-8.86898567e-07 -4.72271501e-07]
F = [2.00301997e-10]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API